In [1]:
%pip install summarytools klib scikit-learn plotly streamlit matplotlib pandas_gbq pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
##############################################################################
                 # Importação de bibliotecas e frameworks # 

import pandas as pd
#import pandas_gbq
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.renderers.default='browser'
template = 'plotly' # "plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white"
import streamlit as st
import klib
import os
import tarfile
import urllib
import sklearn
import urllib.request
from   summarytools import dfSummary
from   zlib import crc32
from   pathlib import Path
from   google.cloud import bigquery
from   google.oauth2 import service_account
from   summarytools import dfSummary
from   sklearn.model_selection import train_test_split

In [3]:
import pkg_resources
def check_package_version(package_name):
    try:
        version = pkg_resources.get_distribution(package_name).version
        return f"{package_name} version: {version}"
    except pkg_resources.DistributionNotFound:
        return f"{package_name} is not installed"

# Check the version of google-cloud package
package_name = "klib"
print(check_package_version(package_name))

klib version: 1.3.1


In [4]:
# Criando uma matriz chamada tabela com as informações do arquivo cancelamentos.csv
tabela = pd.read_csv("large_doencas_brasil.csv")

display(tabela)

,Nome da Doença,Categoria,Região mais Afetada,Incidência Anual (por 100k hab),Faixa Etária Afetada,Mortalidade (%),Número de Casos,Tipo de Tratamento,Custo Médio do Tratamento (R$),Duração Média do Tratamento (dias)
0,HIV/AIDS,Crônica,Sudeste,1629,Todas,15.19,369322,Mudança de Estilo de Vida,28185,287
1,Diabetes,Crônica,Norte,1134,Adolescentes,1.93,408251,Vacina,26502,13
2,Gripe,Infecciosa,Sul,2446,Adolescentes,9.92,429944,Mudança de Estilo de Vida,11315,242
3,Hipertensão,Crônica,Sul,4971,Todas,9.63,330732,Medicamentos,2614,134
4,HIV/AIDS,Infecciosa,Centro-Oeste,2035,Adolescentes,8.78,46147,Medicamentos,507,148
...,...,...,...,...,...,...,...,...,...,...
19995,Tuberculose,Crônica,Sudeste,2350,Crianças,16.20,284085,Vacina,24685,74
19996,Tuberculose,Infecciosa,Norte,4000,Crianças,7.04,341208,Cirurgia,11932,149
19997,Zika,Crônica,Centro-Oeste,1483,Idosos,8.10,419296,Cirurgia,18373,314
19998,Tuberculose,Infecciosa,Sudeste,3549,Todas,5.99,77089,Mudança de Estilo de Vida,10209,347


1. Previsão da Incidência de Doenças por Região
Pergunta: Quais regiões terão maior aumento de casos de determinadas doenças nos próximos meses ou anos?
Aplicação: Modelos de previsão temporal (time-series forecasting) podem ser utilizados para prever a incidência futura de doenças em diferentes regiões, ajudando órgãos de saúde pública a alocar recursos de maneira mais eficaz.
2. Identificação de Fatores Críticos para Mortalidade
Pergunta: Quais fatores estão mais correlacionados com a alta mortalidade de determinadas doenças?
Aplicação: Utilizando técnicas de análise de correlação ou modelos de classificação (como árvores de decisão ou Random Forest), é possível identificar fatores (como idade, região, tratamento utilizado) que mais contribuem para o aumento da mortalidade de doenças.
3. Segmentação de Pacientes para Tratamentos Específicos
Pergunta: Quais grupos de pacientes têm maior probabilidade de responder a determinados tipos de tratamento?
Aplicação: Algoritmos de agrupamento (clustering), como K-means, podem ser usados para identificar perfis de pacientes que respondem melhor a tipos específicos de tratamentos, ajudando a personalizar intervenções médicas.
4. Previsão de Custo de Tratamento
Pergunta: Qual será o custo médio do tratamento para novos casos de doenças com base nos dados históricos?
Aplicação: Modelos de regressão linear ou árvores de regressão podem ser utilizados para prever o custo de tratamentos futuros com base nos fatores de doença, região e incidência.
5. Risco de Doenças por Faixa Etária
Pergunta: Quais faixas etárias têm maior risco de contrair certas doenças em regiões específicas?
Aplicação: Algoritmos de classificação (como Logistic Regression ou Support Vector Machines) podem prever o risco de desenvolvimento de doenças com base na idade e outros fatores demográficos.
6. Otimização de Alocação de Recursos
Pergunta: Como priorizar a alocação de recursos em regiões e doenças específicas para maximizar a eficiência dos tratamentos?
Aplicação: Técnicas de análise de decisão e modelos preditivos podem ajudar a alocar orçamentos e recursos médicos para áreas e doenças com maior demanda, reduzindo custos e otimizando o impacto.
7. Análise da Eficácia de Tratamentos
Pergunta: Qual tratamento tem maior eficácia em termos de redução de casos e mortalidade?
Aplicação: Algoritmos de classificação e análise de sobrevivência podem ser usados para analisar os resultados de diferentes tipos de tratamentos em termos de sua eficácia em reduzir a mortalidade e a duração da doença.
8. Previsão de Picos Epidêmicos
Pergunta: Quando e onde os próximos picos de doenças infecciosas ocorrerão?
Aplicação: Modelos de previsão epidemiológica podem ser construídos para prever picos de doenças como Dengue e Malária, permitindo uma resposta proativa e medidas de contenção.
9. Análise de Custo-Benefício dos Tratamentos
Pergunta: Quais tratamentos têm o melhor custo-benefício em termos de duração e mortalidade reduzida?
Aplicação: Usando modelos de análise de custo-eficácia com os dados de custo médio e duração de tratamentos, pode-se determinar quais opções de tratamento oferecem o maior benefício econômico e de saúde.
10. Análise de Tendências de Doenças
Pergunta: Quais doenças estão em ascensão e quais estão em declínio em diferentes regiões?
Aplicação: Análise de tendências temporais e classificação de séries temporais pode ser utilizada para identificar quais doenças estão em crescimento ou declínio, ajudando a direcionar campanhas de prevenção.